In [ ]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from torchao.dtypes.affine_quantized_tensor import AffineQuantizedTensor
import torch
from torchao.quantization.quant_primitives import MappingType
from torchao.ops import rowwise_scaled_linear_cutlass_s8s4

A = torch.Tensor([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0]]
)

B = torch.Tensor([
    [-2.0, 1.0],
    [3.0, -4.0],
    [6.0, -8.0]]
)

C = torch.Tensor(
    [20.0, -18.0]
)

block_size = [1, 0]
target_dtype = torch.int8
qa = AffineQuantizedTensor.from_hp_to_intx(A, MappingType.ASYMMETRIC, block_size, target_dtype, quant_min=-8, quant_max=7)
block_size = [0, 1]
qb = AffineQuantizedTensor.from_hp_to_intx(B, MappingType.ASYMMETRIC, block_size, target_dtype)

print(qa)
print(qb)

print(rowwise_scaled_linear_cutlass_s8s4(qa.tensor_impl.data, qa.tensor_impl.scale, qb.tensor_impl.data, qb.tensor_impl.scale, None))

TypeError: AffineQuantizedTensor.from_hp_to_intx_static() missing 1 required positional argument: 'target_dtype'

In [ ]:
from torchao.prototype.blockwise_fp8.blockwise_quantization import fp8_blockwise_weight_quant, fp8_blockwise_weight_dequant
from torchao.prototype.blockwise_fp8.blockwise_fp8_gemm_triton import blockwise_fp8_gemm

def test_quant_dequant():
    torch.manual_seed(0)
    x = torch.randn(256, 256)
    qx, s = fp8_blockwise_weight_quant(x, block_size=128)
    x_reconstructed = fp8_blockwise_weight_dequant(qx, s, block_size=128)

    error = torch.norm(x - x_reconstructed) / torch.norm(x)
    print(f"Relative Error: {error.item():.6f}")

    assert error < 0.05, "Quant-Dequant error too high!"

def test_blockwise_fp8_gemm():
    torch.manual_seed(0)
    M, N, K = 256, 256, 256
    A = torch.randn(M, K)
    B = torch.randn(K, N)

    C = A @ B

    A_q, A_s = fp8_blockwise_weight_quant(A, block_size=128)
    B_q, B_s = fp8_blockwise_weight_quant(B, block_size=128)

    C_q = blockwise_fp8_gemm(A_q, A_s, B_q, B_s)

    error = torch.norm(C - C_q) / torch.norm(C)
    print(f"Relative Error: {error.item():.6f}")

    assert error < 0.05, "Quantized GEMM error is too high!"


test_quant_dequant()
test_blockwise_fp8_gemm()

CompilationError: at 1:0:
def fp8_blockwise_quant_weight_kernel(x_ptr, y_ptr, s_ptr, M, N, BLOCK_SIZE: tl.constexpr):
^
ValueError("type fp8e4nv not supported in this architecture. The supported fp8 dtypes are ('fp8e4b15', 'fp8e5')")